In [790]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np
import json
from time import sleep, perf_counter as pf
import itertools as it

In [791]:
def add_type(df, col, val, type):
    df.loc[(df[col.upper()] == val) & (df['TYPE'] == ''), 'TYPE'] = type

In [792]:
def add_cat(df, col, val, type):
    df.loc[(df[col.upper()] == val) & (df['Category'] == ''), 'Category'] = 'Offense Against ' + type

In [793]:
den = pd.read_csv('datasets/Denver.csv')
display(den)

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0
5,201872837,201872837549900,5499,0,traf-other,all-other-crimes,1/31/2018 12:44:00 AM,NaN,1/31/2018 1:29:00 AM,2100 BLOCK E 17TH AVE,3151310.0,1696020.0,-104.961928,39.743149,6,622,city-park-west,1,0
6,20186001015,20186001015230400,2304,0,theft-parts-from-vehicle,theft-from-motor-vehicle,6/1/2017 12:15:00 PM,1/26/2018 12:15:00 PM,1/26/2018 12:24:00 PM,995 N FEDERAL BLVD,3133441.0,1692147.0,-105.025543,39.732790,1,122,villa-park,1,0
7,201870628,201870628570700,5707,0,criminal-trespassing,all-other-crimes,1/30/2018 7:40:00 AM,NaN,1/30/2018 10:33:00 AM,E SPEER BLVD / N GRANT ST,3145202.0,1688799.0,-104.983794,39.723424,3,311,speer,1,0
8,201870833,201870833540100,5401,0,traffic-accident-hit-and-run,traffic-accident,1/30/2018 9:10:00 AM,NaN,1/30/2018 9:17:00 AM,W 13TH AVE / N CHEROKEE ST,3142965.0,1693682.0,-104.991650,39.736863,6,611,civic-center,0,1
9,20186001048,20186001048230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,1/31/2018 12:55:00 AM,1/31/2018 6:55:00 AM,1/31/2018 7:07:00 AM,2828 N ZUNI ST,3136231.0,1701209.0,-105.015451,39.757627,1,113,highland,1,0


In [794]:
van = pd.read_csv('datasets/vancouver.csv')
display(van)

,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,Latitude,Longitude
0,Other Theft,2003,5,12,16.0,15.0,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763
1,Other Theft,2003,5,7,15.0,20.0,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763
2,Other Theft,2003,4,23,16.0,40.0,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763
3,Other Theft,2003,4,20,11.0,15.0,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763
4,Other Theft,2003,4,12,17.0,45.0,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763
5,Other Theft,2003,3,26,20.0,45.0,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763
6,Break and Enter Residential/Other,2003,3,10,12.0,0.0,63XX WILTSHIRE ST,Kerrisdale,489325.58,5452817.95,49.228051,-123.146610
7,Mischief,2003,6,28,4.0,13.0,40XX W 19TH AVE,Dunbar-Southlands,485903.09,5455883.77,49.255559,-123.193725
8,Other Theft,2003,2,16,9.0,2.0,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763
9,Break and Enter Residential/Other,2003,7,9,18.0,15.0,18XX E 3RD AVE,Grandview-Woodland,495078.19,5457221.38,49.267734,-123.067654


In [795]:
# Drop unneeded columns
den.drop(['INCIDENT_ID', 'OFFENSE_ID', 'OFFENSE_CODE', 'OFFENSE_CODE_EXTENSION', 'GEO_X', 'GEO_Y', 'DISTRICT_ID', 'PRECINCT_ID'], 1, inplace=True)

# Add Time Info
datetime = pd.to_datetime(den['REPORTED_DATE'], format='%m/%d/%Y %I:%M:%S %p')
den['DATE'] = [d.date() for d in datetime]
den['TIME'] = [d.time() for d in datetime]

# Add Extra columns
den['City'] = 'Denver'
van['City'] = 'Vancouver'
den['TYPE'] = ''
van['Category'] = ''
den['Category'] = ''

# Add 'IS_TRAFFIC' and 'IS_FATAL'
van['IS_TRAFFIC'] = 0
van.loc[
    (van.TYPE == 'Vehicle Collision or Pedestrian Struck (with Injury)') |
    (van.TYPE == 'Vehicle Collision or Pedestrian Struck (with Fatality)'),
    'IS_TRAFFIC'
] = 1

van['IS_FATAL'] = 0
van.loc[
    (van.TYPE == 'Homicide') |
    (van.TYPE == 'Vehicle Collision or Pedestrian Struck (with Fatality)'),
    'IS_FATAL'
] = 1

den['IS_FATAL'] = 0
den.loc[
    (den.OFFENSE_CATEGORY_ID == 'murder'),
    'IS_FATAL'
] = 1

# Add Start/End time to vancouver
van['FIRST_OCCURRENCE_DATE'] = pd.NaT
van['LAST_OCCURRENCE_DATE'] = pd.NaT
van['Details'] = ''

display(den)

,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_LON,GEO_LAT,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,DATE,TIME,City,TYPE,Category,IS_FATAL
0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,-104.809881,39.773188,montbello,1,0,2016-06-15,23:31:00,Denver,,,0
1,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,-104.781434,39.785649,gateway-green-valley-ranch,1,0,2018-01-29,17:53:00,Denver,,,0
2,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,-104.957381,39.663490,wellshire,1,0,2016-04-26,21:02:00,Denver,,,0
3,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,-104.941440,39.702698,belcaro,1,0,2018-01-30,22:29:00,Denver,,,0
4,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,-104.955370,39.717107,cherry-creek,1,0,2017-06-23,16:09:00,Denver,,,0
5,traf-other,all-other-crimes,1/31/2018 12:44:00 AM,NaN,1/31/2018 1:29:00 AM,2100 BLOCK E 17TH AVE,-104.961928,39.743149,city-park-west,1,0,2018-01-31,01:29:00,Denver,,,0
6,theft-parts-from-vehicle,theft-from-motor-vehicle,6/1/2017 12:15:00 PM,1/26/2018 12:15:00 PM,1/26/2018 12:24:00 PM,995 N FEDERAL BLVD,-105.025543,39.732790,villa-park,1,0,2018-01-26,12:24:00,Denver,,,0
7,criminal-trespassing,all-other-crimes,1/30/2018 7:40:00 AM,NaN,1/30/2018 10:33:00 AM,E SPEER BLVD / N GRANT ST,-104.983794,39.723424,speer,1,0,2018-01-30,10:33:00,Denver,,,0
8,traffic-accident-hit-and-run,traffic-accident,1/30/2018 9:10:00 AM,NaN,1/30/2018 9:17:00 AM,W 13TH AVE / N CHEROKEE ST,-104.991650,39.736863,civic-center,0,1,2018-01-30,09:17:00,Denver,,,0
9,theft-items-from-vehicle,theft-from-motor-vehicle,1/31/2018 12:55:00 AM,1/31/2018 6:55:00 AM,1/31/2018 7:07:00 AM,2828 N ZUNI ST,-105.015451,39.757627,highland,1,0,2018-01-31,07:07:00,Denver,,,0


In [796]:
datetime = pd.to_datetime(van[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']])
van.insert(1, 'DATE', [d.date() for d in datetime])
van.insert(2, 'TIME', [d.time() if not pd.isnull(d) else np.NaN for d in datetime])
van.drop(['X', 'Y', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE'], 1, inplace=True)
display(van)

,TYPE,DATE,TIME,HUNDRED_BLOCK,NEIGHBOURHOOD,Latitude,Longitude,City,Category,IS_TRAFFIC,IS_FATAL,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,Details
0,Other Theft,2003-05-12,16:15:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,,0,0,NaT,NaT,
1,Other Theft,2003-05-07,15:20:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,,0,0,NaT,NaT,
2,Other Theft,2003-04-23,16:40:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,,0,0,NaT,NaT,
3,Other Theft,2003-04-20,11:15:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,,0,0,NaT,NaT,
4,Other Theft,2003-04-12,17:45:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,,0,0,NaT,NaT,
5,Other Theft,2003-03-26,20:45:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,,0,0,NaT,NaT,
6,Break and Enter Residential/Other,2003-03-10,12:00:00,63XX WILTSHIRE ST,Kerrisdale,49.228051,-123.146610,Vancouver,,0,0,NaT,NaT,
7,Mischief,2003-06-28,04:13:00,40XX W 19TH AVE,Dunbar-Southlands,49.255559,-123.193725,Vancouver,,0,0,NaT,NaT,
8,Other Theft,2003-02-16,09:02:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,,0,0,NaT,NaT,
9,Break and Enter Residential/Other,2003-07-09,18:15:00,18XX E 3RD AVE,Grandview-Woodland,49.267734,-123.067654,Vancouver,,0,0,NaT,NaT,


In [797]:
van.TYPE.unique()

array(['Other Theft', 'Break and Enter Residential/Other', 'Mischief',
       'Break and Enter Commercial', 'Offence Against a Person',
       'Theft from Vehicle',
       'Vehicle Collision or Pedestrian Struck (with Injury)',
       'Vehicle Collision or Pedestrian Struck (with Fatality)',
       'Theft of Vehicle', 'Homicide', 'Theft of Bicycle'], dtype=object)

In [798]:
den.OFFENSE_CATEGORY_ID.unique()

array(['all-other-crimes', 'larceny', 'theft-from-motor-vehicle',
       'traffic-accident', 'drug-alcohol', 'auto-theft',
       'white-collar-crime', 'burglary', 'public-disorder',
       'aggravated-assault', 'other-crimes-against-persons', 'robbery',
       'sexual-assault', 'murder', 'arson'], dtype=object)

In [799]:
den.loc[den['OFFENSE_CATEGORY_ID'] == 'murder'].OFFENSE_TYPE_ID.unique()

array(['homicide-other', 'traf-vehicular-homicide', 'homicide-family',
       'homicide-negligent', 'homicide-police-by-gun'], dtype=object)

In [800]:
# Theft
add_type(den, 'offense_category_id', 'theft-from-motor-vehicle', 'Theft from Vehicle')
add_type(den, 'offense_category_id', 'auto-theft', 'Theft of Vehicle')
## Larceny
add_type(den, 'offense_type_id', 'theft-bicycle', 'Theft of Bicycle')
add_type(den, 'offense_category_id', 'larceny', 'Other Theft')
## Robbery
add_type(den, 'offense_type_id', 'robbery-car-jacking', 'Theft of Vehicle')
add_type(den, 'offense_category_id', 'robbery', 'Other Theft')
## Burglary
add_type(den, 'offense_type_id', 'burg-auto-theft-resd-w-force', 'Theft of Vehicle')
add_type(den, 'offense_type_id', 'burg-auto-theft-resd-no-force', 'Theft of Vehicle')
add_type(den, 'offense_type_id', 'burg-auto-theft-busn-w-force', 'Theft of Vehicle')
add_type(den, 'offense_type_id', 'burg-auto-theft-busn-no-force', 'Theft of Vehicle')
add_type(den, 'offense_category_id', 'burglary', 'Other Theft')
## White-collar-crimes
add_type(den, 'offense_type_id', 'theft-embezzle', 'Other Theft')
add_type(den, 'offense_type_id', 'theft-unauth-use-of-ftd', 'Other Theft')
add_type(den, 'offense_type_id', 'theft-fail-return-rent-veh', 'Theft of Vehicle')
add_type(den, 'offense_type_id', 'theft-of-rental-property', 'Other Theft') 

# Homocide
add_type(den, 'offense_type_id', 'traf-vehicular-homicide', 'Vehicle Collision or Pedestrian Struck (with Fatality)')
add_type(den, 'offense_category_id', 'murder', 'Homicide')
add_type(den, 'offense_category_id', 'traffic-accident', 'Vehicle Collision or Pedestrian Struck (with Injury)')

# Public Disorder 
add_type(den, 'offense_type_id', 'criminal-mischief-other', 'Mischief')
add_type(den, 'offense_type_id', 'criminal-mischief-mtr-veh', 'Mischief')
add_type(den, 'offense_type_id', 'criminal-mischief-graffiti', 'Mischief')
add_type(den, 'offense_type_id', 'riot', 'Mischief')
add_type(den, 'offense_type_id', 'loitering', 'Mischief')
add_type(den, 'offense_type_id', 'prostitution-engaging-in', 'Mischief')
add_type(den, 'offense_type_id', 'prostitution-aiding', 'Mischief')
add_type(den, 'offense_category_id', 'public-disorder', 'Offence Against a Person')

#all-other-crimes
add_type(den, 'offense_type_id', 'criminal-trespassing', 'Mischief')
add_type(den, 'offense_type_id', 'criminal-trespassing', 'Mischief')
add_type(den, 'offense_type_id', 'littering', 'Mischief') 
add_type(den, 'offense_type_id', 'public-fighting', 'Mischief')
add_type(den, 'offense_type_id', 'illegal-dumping', 'Mischief')
add_type(den, 'offense_type_id', 'eavesdropping', 'Mischief')

# aggravated
add_type(den, 'offense_category_id', 'aggravated-assault', 'Offence Against a Person')
add_type(den, 'offense_category_id', 'sexual-assault', 'Offence Against a Person')
add_type(den, 'offense_category_id', 'other-crimes-against-persons', 'Offence Against a Person')

display(den.loc[(den['TYPE'] == '') & (~den.OFFENSE_CATEGORY_ID.isin([
    'all-other-crimes',
    'drug-alcohol',
    'white-collar-crime',
    'arson',
]))])

,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_LON,GEO_LAT,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,DATE,TIME,City,TYPE,Category,IS_FATAL


In [801]:
add_cat(den, 'type', 'Theft from Vehicle', 'Person')
add_cat(van, 'type', 'Theft from Vehicle', 'Person')
add_cat(den, 'type', 'Other Theft', 'Property')
add_cat(van, 'type', 'Other Theft', 'Property')
add_cat(den, 'type', 'Theft of Vehicle', 'Property')
add_cat(van, 'type', 'Theft of Vehicle', 'Property')
add_cat(den, 'type', 'Theft of Bicycle', 'Property')
add_cat(van, 'type', 'Theft of Bicycle', 'Property')
add_cat(den, 'type', 'Offence Against a Person', 'Person')
add_cat(van, 'type', 'Offence Against a Person', 'Person')
add_cat(den, 'type', 'Vehicle Collision or Pedestrian Struck (with Injury)', 'Society')
add_cat(van, 'type', 'Vehicle Collision or Pedestrian Struck (with Injury)', 'Society')
add_cat(den, 'type', 'Vehicle Collision or Pedestrian Struck (with Fatality)', 'Society')
add_cat(van, 'type', 'Vehicle Collision or Pedestrian Struck (with Fatality)', 'Society')
add_cat(den, 'type', 'Homicide', 'Person')
add_cat(van, 'type', 'Homicide', 'Person')
add_cat(den, 'type', 'Mischief', 'Society')
add_cat(van, 'type', 'Mischief', 'Society')
add_cat(den, 'type', 'Break and Enter Residential/Other', 'Property')
add_cat(van, 'type', 'Break and Enter Residential/Other', 'Property')
add_cat(den, 'type', 'Break and Enter Commercial', 'Property')
add_cat(van, 'type', 'Break and Enter Commercial', 'Property')

add_cat(den, 'offense_category_id', 'all-other-crimes', 'Society')
add_cat(den, 'offense_category_id', 'aggravated-assault', 'Person')
add_cat(den, 'offense_category_id', 'sexual-assault', 'Person')
add_cat(den, 'offense_category_id', 'white-collar-crime', 'Society')
add_cat(den, 'offense_category_id', 'arson', 'Property')
add_cat(den, 'offense_category_id', 'drug-alcohol', 'Society')

display(den.loc[den['Category'] == ''])
display(van.loc[van['Category'] == ''])

,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_LON,GEO_LAT,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,DATE,TIME,City,TYPE,Category,IS_FATAL


,TYPE,DATE,TIME,HUNDRED_BLOCK,NEIGHBOURHOOD,Latitude,Longitude,City,Category,IS_TRAFFIC,IS_FATAL,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,Details


In [802]:
display(van)

,TYPE,DATE,TIME,HUNDRED_BLOCK,NEIGHBOURHOOD,Latitude,Longitude,City,Category,IS_TRAFFIC,IS_FATAL,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,Details
0,Other Theft,2003-05-12,16:15:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,Offense Against Property,0,0,NaT,NaT,
1,Other Theft,2003-05-07,15:20:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,Offense Against Property,0,0,NaT,NaT,
2,Other Theft,2003-04-23,16:40:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,Offense Against Property,0,0,NaT,NaT,
3,Other Theft,2003-04-20,11:15:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,Offense Against Property,0,0,NaT,NaT,
4,Other Theft,2003-04-12,17:45:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,Offense Against Property,0,0,NaT,NaT,
5,Other Theft,2003-03-26,20:45:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,Offense Against Property,0,0,NaT,NaT,
6,Break and Enter Residential/Other,2003-03-10,12:00:00,63XX WILTSHIRE ST,Kerrisdale,49.228051,-123.146610,Vancouver,Offense Against Property,0,0,NaT,NaT,
7,Mischief,2003-06-28,04:13:00,40XX W 19TH AVE,Dunbar-Southlands,49.255559,-123.193725,Vancouver,Offense Against Society,0,0,NaT,NaT,
8,Other Theft,2003-02-16,09:02:00,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,Vancouver,Offense Against Property,0,0,NaT,NaT,
9,Break and Enter Residential/Other,2003-07-09,18:15:00,18XX E 3RD AVE,Grandview-Woodland,49.267734,-123.067654,Vancouver,Offense Against Property,0,0,NaT,NaT,


In [803]:
den.drop(['OFFENSE_TYPE_ID', 'REPORTED_DATE', 'IS_CRIME'], axis=1, inplace=True)
display(den)

,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,INCIDENT_ADDRESS,GEO_LON,GEO_LAT,NEIGHBORHOOD_ID,IS_TRAFFIC,DATE,TIME,City,TYPE,Category,IS_FATAL
0,all-other-crimes,6/15/2016 11:31:00 PM,NaN,NaN,-104.809881,39.773188,montbello,0,2016-06-15,23:31:00,Denver,,Offense Against Society,0
1,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,NaN,-104.781434,39.785649,gateway-green-valley-ranch,0,2018-01-29,17:53:00,Denver,Other Theft,Offense Against Property,0
2,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,2932 S JOSEPHINE ST,-104.957381,39.663490,wellshire,0,2016-04-26,21:02:00,Denver,Theft from Vehicle,Offense Against Person,0
3,larceny,1/30/2018 7:20:00 PM,NaN,705 S COLORADO BLVD,-104.941440,39.702698,belcaro,0,2018-01-30,22:29:00,Denver,Other Theft,Offense Against Property,0
4,larceny,6/22/2017 8:53:00 PM,NaN,2810 E 1ST AVE,-104.955370,39.717107,cherry-creek,0,2017-06-23,16:09:00,Denver,Other Theft,Offense Against Property,0
5,all-other-crimes,1/31/2018 12:44:00 AM,NaN,2100 BLOCK E 17TH AVE,-104.961928,39.743149,city-park-west,0,2018-01-31,01:29:00,Denver,,Offense Against Society,0
6,theft-from-motor-vehicle,6/1/2017 12:15:00 PM,1/26/2018 12:15:00 PM,995 N FEDERAL BLVD,-105.025543,39.732790,villa-park,0,2018-01-26,12:24:00,Denver,Theft from Vehicle,Offense Against Person,0
7,all-other-crimes,1/30/2018 7:40:00 AM,NaN,E SPEER BLVD / N GRANT ST,-104.983794,39.723424,speer,0,2018-01-30,10:33:00,Denver,Mischief,Offense Against Society,0
8,traffic-accident,1/30/2018 9:10:00 AM,NaN,W 13TH AVE / N CHEROKEE ST,-104.991650,39.736863,civic-center,1,2018-01-30,09:17:00,Denver,Vehicle Collision or Pedestrian Struck (with I...,Offense Against Society,0
9,theft-from-motor-vehicle,1/31/2018 12:55:00 AM,1/31/2018 6:55:00 AM,2828 N ZUNI ST,-105.015451,39.757627,highland,0,2018-01-31,07:07:00,Denver,Theft from Vehicle,Offense Against Person,0


In [804]:
van.columns=['TYPE', 'DATE', 'TIME', 'ADDRESS', 'NEIGHBORHOOD', 'LAT', 'LON', 'City', 'Category', 'IS_TRAFFIC', 'IS_FATAL', 'start_dt', 'end_dt', 'Details']
den.columns=['Details', 'start_dt', 'end_dt', 'ADDRESS', 'LON', 'LAT', 'NEIGHBORHOOD', 'IS_TRAFFIC', 'DATE', 'TIME', 'City', 'TYPE', 'Category', 'IS_FATAL']
crime = pd.concat([van, den], sort=True)
crime.end_dt=pd.to_datetime(crime.end_dt)
crime.start_dt=pd.to_datetime(crime.start_dt)
crime = crime.loc[(crime.DATE >= '2015/01/01') & (crime.DATE <= '2017/01/01')]
display(crime)

TypeError: '>=' not supported between instances of 'datetime.date' and 'str'

In [ ]:
crime.reset_index(drop=True, inplace=True) # If not we get non-unique keys
crime.insert(0, 'Crime_key', crime.index)
display(crime)

In [ ]:
crime.to_csv('datasets/crime_output.csv', index=False)

In [ ]:
# Get Final Crime Table
f_crime = crime.drop(['ADDRESS', 'City', 'DATE', 'IS_FATAL', 'IS_TRAFFIC', 'LAT', 'LON', 'NEIGHBORHOOD'], axis=1)
f_crime.to_csv('datasets/final/crime.csv', index=False)
display(f_crime)

In [ ]:
location = crime[['ADDRESS', 'LAT', 'LON', 'NEIGHBORHOOD']]
location = location.drop_duplicates()
location.reset_index(drop=True, inplace=True)
location.insert(0, 'Location_key', location.index)
display(location)

In [ ]:
location.to_csv('datasets/location_output.csv', index=False)